In [1]:
sc

In [3]:

import json
import datetime
import sys
import time
import pyspark
from operator import add
import numpy as np
import matplotlib.path as mplPath

In [7]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def find_Block(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

In [ ]:
def mapToZone(stations):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('block-groups-polygons.geojson')
    for station in stations[1:]:
        station = station.strip('').split(',')
        location = geom.Point(proj(station[5], station[4]))
        zone = find_Block(location, index, zones)
        yield (station[1], zone)

stations = sc.textFile('../CitiBike.csv')
output = mapToZone(stations.collect()) # station, blockid
li = list(output)

In [ ]:
import json
coordinatesList = []
with open ('block-groups-polygons.geojson') as dataFile:
    blockData = json.load(dataFile)
for k,v in li:
    for block in blockData['features']:
        if v == block['properties']['OBJECTID']:
            coordinatesList.append(block['geometry']['coordinates'])
            #print (k,v,block['geometry']['coordinates']) 

coordinatesList